<h1>Objetos (básicamente todo en Python es un objeto!)</h1>

Cuando asignamos valores entre variables **nunca se efectúa una copia** del valor, sino sus referencias (*punteros*) hacia la dirección de memoria donde se aloja el valor.

En el ejemplo siguiente vemos que esto no pasa

In [6]:
a = 50
b = a
print(a, id(a), b, id(b))
a = 100
print(a, b)

50 1669724864 50 1669724864
100 50


Sin embargo, cuando hablamos de listas...

In [21]:
a = [1, 2, 3]
b = a
print(a is b)
c = [a, b]
print(a, b, c)
a.append(100)
print(a, b, c)
print(id(a), id(b), id(c[0]), id(c[1]))

True
[1, 2, 3] [1, 2, 3] [[1, 2, 3], [1, 2, 3]]
[1, 2, 3, 100] [1, 2, 3, 100] [[1, 2, 3, 100], [1, 2, 3, 100]]
99648648 99648648 99648648 99648648


Las listas y los diccionarios tienen métodos para hacer copias reales de los valores.

In [38]:
a2 = [2, 3, [100, 101], 4]
b2 = list(a)
a2 is b2

False

Vemos que 'b' es una lista nueva, con un valor de `id` distinto del de a

In [39]:
a2.insert(0, 1)
print(a2)
print(b2)

[1, 2, 3, [100, 101], 4]
[2, 3, [100, 101], 4]


Sin embargo, veamos algo curioso.

In [41]:
a1 = [1, 2, [100, 101], 3, 4]
b1 = list(a1)
a1.append(5)
print(a1)
print(b1)
a1[2].append(102)
print(a1)
print(b1)

[1, 2, [100, 101], 3, 4, 5]
[1, 2, [100, 101], 3, 4]
[1, 2, [100, 101, 102], 3, 4, 5]
[1, 2, [100, 101, 102], 3, 4]


Vemos que la **lista interna** seguía estando compartida en cierta forma!<br>
A esto se le llama *shallow copy* o copia superficial. Veamos que podemos hacer copias profundas (*deep copy*)

In [44]:
import copy

a3 = [1, 2, [100, 101], 3, 4]
b3 = copy.deepcopy(a3)
a3.append(5)
print(a1)
print(b1)
a3[2].append(102)
print(a3)
print(b3)

[1, 2, [100, 101, 102], 3, 4, 5]
[1, 2, [100, 101, 102], 3, 4]
[1, 2, [100, 101, 102], 3, 4, 5]
[1, 2, [100, 101], 3, 4]


Acá se realizó una copia profunda (hasta el último nivel de profundidad en la variabla a3)

<h2> Verificación de tipos </h2>

Tenemos un par de funciones copadas para corroborar tipo de datos

In [46]:
a = [1, 2, 3]
isinstance(a, list)

True

In [47]:
b = {}
isinstance(b, (list, tuple, dict))

True

<h2> Todo es un objeto </h2>

Números, cadenas, listas, funciones, excepciones, clases, instancias, etc. son todos objetos. Esto significa que pueden ser nombrados, pueden ser pasados como datos, ubicados en contenedores, etc. sin restricciones. No hay objetos especiales en Python.

In [48]:
import math
items = [abs, math, ValueError]
print(items)

[<built-in function abs>, <module 'math' (built-in)>, <class 'ValueError'>]


In [49]:
items[0](-45)

45

In [50]:
items[1].sqrt(2)

1.4142135623730951


Zarpado en loco! Podemos armar una lista de una funcion, un módulo y una excepción...<br>
Pero como día Uncle Ben: '*Remember, with great power comes great responsibility.*'

Qué se puede hacer con esto? Podríamos hacer una conversión de tipos de manera loca.
<br>
Por ejemplo, en nuestro caso del camión de frutas. Teníamos como columnas **nombre**: `string`, **cajones**: `int` y **precio**: `float`.
<br>
Podemos hacer algunos truquitos de magia

Primero vamos a levantar el archivo y luego vamos a jugar un poco con él

In [54]:
import csv
nombre_archivo = '../Data/camion.csv'

types = [str, int, float]

with open(nombre_archivo, 'rt') as file:
    filas = csv.reader(file)
    headers = next(filas)
    fila = next(filas)
    fila_piola = list(zip(types, fila))
# Acá vemos la fila común, todos strings
print(fila)
# Acá vemos (sin ser útil) que nos queda apareados los tipos y los datos
print(fila_piola)

### LO ÚTIL Y COPADO ###
convertidos = []
for func, val in zip(types, fila):
    convertidos.append(func(val))
print(convertidos)

['Lima', '100', '32.2']
[(<class 'str'>, 'Lima'), (<class 'int'>, '100'), (<class 'float'>, '32.2')]
['Lima', 100, 32.2]


O mejor aún, si queremos usar comprensión de listas:

In [56]:
convertidos2 = [func(val) for func, val in zip(types, fila)]
convertidos2

['Lima', 100, 32.2]

**Hermoso**

Y podemos ir un pasito más allá. Podemos armar un diccionario muy fácilmente ahora que tenemos las líneas bien identificadas con sus tipos (antes también podíamos pero había que convertirlas una a una para que quedara bien)

In [57]:
diccio = dict(zip(headers, convertidos2))
diccio

{'nombre': 'Lima', 'cajones': 100, 'precio': 32.2}

O, todo en una sola línea usando comprensión de diccio:

In [59]:
diccio1 = {nombre: func(val) for nombre, func, val in zip(headers, types, fila)}
diccio1

{'nombre': 'Lima', 'cajones': 100, 'precio': 32.2}

Vamos con un último ejemplo, que junta todo esto.

In [62]:
nombre_archivo = '../Data/dowstocks.csv'
with open(nombre_archivo, 'rt') as file:
    filas = csv.reader(file)
    headers = next(filas)
    fila = next(filas)
    print(headers)
    print(fila)

['name', 'price', 'date', 'time', 'change', 'open', 'high', 'low', 'volume']
['AA', '39.48', '6/11/2007', '9:36am', '-0.18', '39.67', '39.69', '39.45', '181800']


In [65]:
types = [str, float, str, str, float, float, float, float, int]
converted = [func(val) for func, val in zip(types, fila)]
converted

['AA', 39.48, '6/11/2007', '9:36am', -0.18, 39.67, 39.69, 39.45, 181800]

In [66]:
record = dict(zip(headers, converted))
record

{'name': 'AA',
 'price': 39.48,
 'date': '6/11/2007',
 'time': '9:36am',
 'change': -0.18,
 'open': 39.67,
 'high': 39.69,
 'low': 39.45,
 'volume': 181800}

Muy piola! Y ahora, para armar una lista con todos los datos del archivo (lista de dict)

In [70]:
import csv
from pprint import pprint as pp
nombre_archivo = '../Data/dowstocks.csv'
with open(nombre_archivo, 'rt') as file:
    filas = csv.reader(file)
    headers = next(filas)
    types = [str, float, str, str, float, float, float, float, int]
    lista = [{name: func(val) for name, func, val in zip(headers, types, fila)} for fila in filas]
with open('archivo.txt', 'wt') as archivo_salida:
    print(lista, file = archivo_salida)

In [72]:
lista[0:5]

[{'name': 'AA',
  'price': 39.48,
  'date': '6/11/2007',
  'time': '9:36am',
  'change': -0.18,
  'open': 39.67,
  'high': 39.69,
  'low': 39.45,
  'volume': 181800},
 {'name': 'AIG',
  'price': 71.38,
  'date': '6/11/2007',
  'time': '9:36am',
  'change': -0.15,
  'open': 71.29,
  'high': 71.6,
  'low': 71.15,
  'volume': 195500},
 {'name': 'AXP',
  'price': 62.58,
  'date': '6/11/2007',
  'time': '9:36am',
  'change': -0.46,
  'open': 62.79,
  'high': 63.0,
  'low': 62.57,
  'volume': 935000},
 {'name': 'BA',
  'price': 98.31,
  'date': '6/11/2007',
  'time': '9:36am',
  'change': 0.12,
  'open': 98.25,
  'high': 98.58,
  'low': 98.19,
  'volume': 104800},
 {'name': 'C',
  'price': 53.08,
  'date': '6/11/2007',
  'time': '9:36am',
  'change': -0.25,
  'open': 53.2,
  'high': 53.25,
  'low': 53.03,
  'volume': 360900}]

In [78]:
nombres = [dato['name'] for dato in lista]
print(len(nombres))
print(len(set(nombres)))
print(sorted(set(nombres)))

2340
30
['AA', 'AIG', 'AXP', 'BA', 'C', 'CAT', 'DD', 'DIS', 'GE', 'GM', 'HD', 'HON', 'HPQ', 'IBM', 'INTC', 'JNJ', 'JPM', 'KO', 'MCD', 'MMM', 'MO', 'MRK', 'MSFT', 'PFE', 'PG', 'T', 'UTX', 'VZ', 'WMT', 'XOM']
